In [119]:
import pandas as pd
import numpy as np

In [152]:
bb_train = pd.read_csv('../data/Annotation/boundingbox/bb_train.csv', index_col='id')
bb_test = pd.read_csv('../data/Annotation/boundingbox/bb_test.csv', index_col='id')
# manual_train = pd.read_csv('../data/Annotation/manual/filled_manual_train.csv', index_col='id')
# manual_test = pd.read_csv('../data/Annotation/manual/filled_manual_test.csv', index_col='id')
manual_train = pd.read_csv('../data/Annotation/auto/auto_train.csv', index_col='id')
manual_test = pd.read_csv('../data/Annotation/auto/auto_test.csv', index_col='id')

In [153]:
train_join = manual_train.join(bb_train)
test_join = manual_test.join(bb_test)

In [154]:
train_join

,auto_1_X,auto_1_Y,auto_2_X,auto_2_Y,auto_3_X,auto_3_Y,auto_4_X,auto_4_Y,auto_5_X,auto_5_Y,...,auto_35_X,auto_35_Y,auto_36_X,auto_36_Y,auto_37_X,auto_37_Y,bb_1_pt1,bb_1_pt2,bb_1_pt3,bb_1_pt4
id,,,,,,,,,,,,,,,,,,,,,
8207,416.0,433.0,459.0,413.5,504.0,430.0,589.0,436.0,658.5,440.5,...,497.0,715.0,469.0,703.0,455.0,673.0,400.727661,279.273773,786.620544,802.815247
8212,356.0,511.0,453.0,477.0,540.0,509.0,640.0,516.0,704.0,488.5,...,603.0,836.0,550.0,831.0,503.0,813.0,224.027008,317.896545,741.898987,1020.551636
2168,133.0,169.0,153.0,156.5,176.0,159.0,206.0,159.0,226.0,154.5,...,203.0,242.0,190.0,241.0,178.0,236.0,120.546776,108.693390,246.159988,272.392456
7952,145.0,167.0,156.5,164.0,168.0,167.0,186.0,176.0,200.0,176.5,...,167.0,224.0,163.0,223.0,161.0,219.0,122.614677,108.932533,259.113892,275.525970
7947,91.0,103.0,107.5,100.0,121.0,102.0,138.0,100.0,149.5,96.5,...,128.0,164.0,122.0,163.0,117.0,160.0,72.309723,70.940666,162.756195,176.609451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,124.0,159.0,139.0,148.0,160.0,146.0,186.0,144.0,209.0,143.0,...,179.0,231.0,172.0,231.0,165.0,230.0,115.589203,104.835175,236.395050,261.911621
9580,31.0,39.0,35.0,36.0,41.0,35.0,49.0,33.0,53.0,32.0,...,52.0,52.0,50.0,52.0,47.0,53.0,27.461517,26.941301,61.796883,63.526768
4031,330.0,425.0,389.5,430.0,458.0,447.0,566.0,445.0,635.5,417.0,...,538.0,694.0,506.0,688.0,488.0,676.0,316.955902,237.855606,753.357483,757.913391


In [138]:
# num_landmarks = 5
# X_cols = [f'manual_{i}_X' for i in range(1, num_landmarks + 1)]
# Y_cols = [f'manual_{i}_Y' for i in range(1, num_landmarks + 1)]

# categorical_cols = ['Gender', 'Race', 'Age']

# def standardize(df):
#     X_df = df[X_cols].subtract(df['bb_1_pt1'], axis=0).divide(df['bb_1_pt3'] - df['bb_1_pt1'], axis=0)
#     Y_df = df[Y_cols].subtract(df['bb_1_pt2'], axis=0).divide(df['bb_1_pt4'] - df['bb_1_pt2'], axis=0)
#     categorical_df = df[categorical_cols]
#     return X_df.join(Y_df).join(categorical_df)

In [155]:
num_landmarks = 37
X_cols = [f'auto_{i}_X' for i in range(1, num_landmarks + 1)]
Y_cols = [f'auto_{i}_Y' for i in range(1, num_landmarks + 1)]

def standardize(df):
    X_df = df[X_cols].subtract(df['bb_1_pt1'], axis=0).divide(df['bb_1_pt3'] - df['bb_1_pt1'], axis=0)
    Y_df = df[Y_cols].subtract(df['bb_1_pt2'], axis=0).divide(df['bb_1_pt4'] - df['bb_1_pt2'], axis=0)
    return X_df.join(Y_df)

In [156]:
std_train = standardize(train_join)
std_test = standardize(test_join)

In [157]:
def sq_euclidean_distance(x1, y1, x2, y2):
    return (x1 - x2) ** 2 + (y1 - y2) ** 2

In [158]:
for i in range(num_landmarks):
    for j in range(i + 1, num_landmarks):
        x1_col = X_cols[i]
        y1_col = Y_cols[i]
        x2_col = X_cols[j]
        y2_col = Y_cols[j]
        new_col_name = f'{i+1}_{j+1}_sq_dist'
        std_train[new_col_name] = sq_euclidean_distance(std_train[x1_col],
                                                        std_train[y1_col], 
                                                        std_train[x2_col],
                                                        std_train[y2_col])
        std_test[new_col_name] = sq_euclidean_distance(std_test[x1_col], 
                                                       std_test[y1_col],
                                                       std_test[x2_col], 
                                                       std_test[y2_col])

/var/folders/tk/xgt1qkjn2250kh87lr8djx740000gn/T/ipykernel_75565/1645323454.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  std_train[new_col_name] = sq_euclidean_distance(std_train[x1_col],
/var/folders/tk/xgt1qkjn2250kh87lr8djx740000gn/T/ipykernel_75565/1645323454.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  std_test[new_col_name] = sq_euclidean_distance(std_test[x1_col],
/var/folders/tk/xgt1qkjn2250kh87lr8djx740000gn/T/ipykernel_75565/1645323454.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [159]:
std_test

,auto_1_X,auto_2_X,auto_3_X,auto_4_X,auto_5_X,auto_6_X,auto_7_X,auto_8_X,auto_9_X,auto_10_X,...,32_33_sq_dist,32_34_sq_dist,32_35_sq_dist,32_36_sq_dist,33_34_sq_dist,33_35_sq_dist,33_36_sq_dist,34_35_sq_dist,34_36_sq_dist,35_36_sq_dist
id,,,,,,,,,,,,,,,,,,,,,
2372,0.111492,0.196860,0.298751,0.474995,0.615439,0.739361,0.370350,0.238167,0.166568,0.205122,...,0.009965,0.029151,0.044046,0.060249,0.005263,0.012880,0.023079,0.001730,0.006649,0.001640
2367,0.262327,0.394302,0.536238,0.740426,0.867421,0.969515,0.635842,0.685644,0.461535,0.546198,...,0.002706,0.012512,0.039909,0.075027,0.003628,0.022322,0.050450,0.008092,0.027517,0.005806
433,0.026816,0.161895,0.296974,0.567131,0.729226,0.891321,0.432052,0.405037,0.283466,0.351005,...,0.012704,0.049561,0.091162,0.143165,0.012134,0.036219,0.073441,0.006569,0.028099,0.008393
426,0.120077,0.294947,0.447958,0.655616,0.819557,0.961639,0.546323,0.546323,0.393311,0.458888,...,0.008666,0.027241,0.065143,0.093728,0.005931,0.030658,0.056507,0.009988,0.028413,0.005799
982,0.121932,0.295861,0.440802,0.658214,0.803154,0.933601,0.542261,0.527767,0.382826,0.455296,...,0.005765,0.021521,0.053908,0.093158,0.005252,0.025548,0.055832,0.007691,0.027472,0.006406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,0.242830,0.402254,0.561678,0.764581,0.895019,0.996471,0.677623,0.779075,0.576171,0.677623,...,0.004435,0.019957,0.053775,0.103293,0.006167,0.031163,0.075930,0.010292,0.042798,0.011921
1859,0.109054,0.255258,0.401461,0.610323,0.798299,0.923616,0.505892,0.464120,0.338803,0.401461,...,0.004144,0.021593,0.043623,0.073941,0.006980,0.021593,0.044495,0.004144,0.016576,0.004144
2411,0.233196,0.108871,0.185547,0.355330,0.475821,0.601789,0.262223,0.262223,0.245793,0.273177,...,0.002464,0.008454,0.013954,0.015297,0.001792,0.004724,0.005724,0.000791,0.001566,0.000252


In [160]:
#read true labels
labels = pd.read_table('../data/basic/EmoLabel/list_patition_label.txt', sep=' ', header=None)

In [161]:
#separate train and test labels
train_labels = {}
test_labels = {}
for index, row in labels.iterrows():
    if 'train' in row[0]:
        split_id = row[0].split('_')[1].split('.')[0]
        train_labels.update({int(split_id): row[1]})
    elif 'test' in row[0]:
        split_id = row[0].split('_')[1].split('.')[0]
        test_labels.update({int(split_id): row[1]})
        
train_classes = [-1]*len(train_labels)
for k, v in train_labels.items():
    train_classes[int(k)-1] = v

test_classes = [-1]*len(test_labels)
for k, v in test_labels.items():
    test_classes[int(k)-1] = v

In [162]:
len(train_labels)

12271

In [163]:
len(test_labels)

3068

In [164]:
std_train['label'] = pd.Series(train_labels)
std_test['label'] = pd.Series(test_labels)

/var/folders/tk/xgt1qkjn2250kh87lr8djx740000gn/T/ipykernel_75565/1554946161.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  std_train['label'] = pd.Series(train_labels)
/var/folders/tk/xgt1qkjn2250kh87lr8djx740000gn/T/ipykernel_75565/1554946161.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  std_test['label'] = pd.Series(test_labels)


In [148]:
# std_train_dummies = pd.get_dummies(std_train, columns=['Gender', 'Race', 'Age'])
# std_test_dummies = pd.get_dummies(std_test, columns=['Gender', 'Race', 'Age'])

In [165]:
from sklearn.linear_model import LogisticRegression

In [150]:
clf = LogisticRegression(max_iter=500).fit(std_train_dummies.drop(columns=['label']),
                               std_train_dummies['label'])

In [151]:
clf.score(std_test_dummies.drop(columns=['label']), std_test_dummies['label'])

0.5573663624511083

In [166]:
clf = LogisticRegression(max_iter=1000).fit(std_train.drop(columns=['label']),
                               std_train['label'])

In [167]:
clf.score(std_test.drop(columns=['label']), std_test['label'])

0.636245110821382

In [134]:
# std_train.to_csv('../data/Annotation/auto/auto_std_train.csv')
# std_test.to_csv('../data/Annotation/auto/auto_std_test.csv')

In [ ]:
# std_train.to_csv('../data/Annotation/manual/manual_std_train.csv')
# std_test.to_csv('../data/Annotation/manual/manual_std_test.csv')